In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the dataset for the teaching building
teaching_train_df = pd.read_csv('building_type_teaching_kaggle_train_dataset.csv')

# Extract relevant features and target
features = teaching_train_df[['ApparentTemperature', 'RelativeHumidity', 'CDD', 'HDD', 'Hour24', 'Minute']]  # Example features
target = teaching_train_df['MeterReading']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Support Vector Regressor model
svr_model = SVR()
svr_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = svr_model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')

# Save the trained model and scaler
with open('teaching_svr_model.pkl', 'wb') as model_file:
    pickle.dump(svr_model, model_file)
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

print("Model and scaler have been saved.")

Mean Absolute Error: 2.298218847715843
Mean Squared Error: 9.011497157207764
Root Mean Squared Error: 3.001915581292679
Model and scaler have been saved.


In [2]:
!pip install tensorflow

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
import numpy as np

# Load the training dataset for the teaching building
teaching_train_df = pd.read_csv('building_type_teaching_kaggle_train_dataset.csv')

# Extract relevant features and target
features = teaching_train_df[['ApparentTemperature', 'RelativeHumidity', 'CDD', 'HDD', 'Hour24', 'Minute']]  # Example features
target = teaching_train_df['MeterReading']

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(features)
y_train = target

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Save the trained model in SavedModel format
model.save('teaching_nn_model')

# Save the scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

# Predict on the training set
y_train_pred = model.predict(X_train_scaled)

# Calculate and print the evaluation metrics
mae = mean_absolute_error(y_train, y_train_pred)
mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

print("Neural network model and scaler have been saved.")

Epoch 1/20
728/728 [==============================] - 6s 5ms/step - loss: 38.2936 - val_loss: 14.4718
Epoch 2/20
728/728 [==============================] - 3s 5ms/step - loss: 10.9465 - val_loss: 12.4027
Epoch 3/20
728/728 [==============================] - 5s 6ms/step - loss: 9.3292 - val_loss: 13.1718
Epoch 4/20
728/728 [==============================] - 5s 6ms/step - loss: 8.7418 - val_loss: 13.3150
Epoch 5/20
728/728 [==============================] - 3s 4ms/step - loss: 8.3138 - val_loss: 11.8125
Epoch 6/20
728/728 [==============================] - 3s 5ms/step - loss: 7.9593 - val_loss: 11.7849
Epoch 7/20
728/728 [==============================] - 5s 7ms/step - loss: 7.6410 - val_loss: 12.0589
Epoch 8/20
728/728 [==============================] - 1s 2ms/step - loss: 7.3985 - val_loss: 11.8571
Epoch 9/20
728/728 [==============================] - 1s 2ms/step - loss: 7.1817 - val_loss: 12.8388
Epoch 10/20
728/728 [==============================] - 1s 2ms/step - loss: 7.0245 - val_l

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import pickle
import tensorflow as tf

# Load the dataset for the teaching building
teaching_df = pd.read_csv('building_type_teaching_kaggle_train_dataset.csv')

# Convert the Timestamp to datetime format and extract the date and hour
teaching_df['Timestamp'] = pd.to_datetime(teaching_df['Timestamp'])
teaching_df['Date'] = teaching_df['Timestamp'].dt.date
teaching_df['Hour24'] = teaching_df['Timestamp'].dt.hour
teaching_df['Minute'] = teaching_df['Timestamp'].dt.minute

# Load the trained neural network model and scaler
nn_model = tf.keras.models.load_model('teaching_nn_model')
with open('scaler.pkl', 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)

# Define the energy usage limit (example value)
ENERGY_USAGE_LIMIT = 5  # kWh

# Define the default current time
current_time = pd.to_datetime('2018-10-31 23:45')

def summarize_usage_patterns(date_str='2018-10-31'):
    # Convert the date string to a datetime object
    specific_date = pd.to_datetime(date_str).date()

    # Filter data for the specific date
    df_specific_date = teaching_df[teaching_df['Date'] == specific_date]

    # Calculate average MeterReading based on 'Hour24'
    avg_meter_reading_hourly = df_specific_date.groupby('Hour24')['MeterReading'].mean()

    # Convert to numpy arrays for plotting
    hours = avg_meter_reading_hourly.index.to_numpy()
    avg_meter_readings = avg_meter_reading_hourly.to_numpy()

    # Create the plot
    plt.figure(figsize=(14, 7))
    plt.plot(hours, avg_meter_readings, label=f'Teaching Building - {specific_date}', marker='o')

    plt.title(f'Average MeterReading by Hour for Teaching Building on {specific_date}')
    plt.xlabel('Hour (24-hour format)')
    plt.ylabel('Average Meter Reading')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.xticks(range(0, 24))
    plt.show()

    return f"I've displayed the average MeterReading by hour for the Teaching Building on {specific_date}."

def predict_energy_consumption(period_minutes):
    # Generate future time points based on the period
    future_times = [current_time + timedelta(minutes=i) for i in range(0, period_minutes + 1, 15)]

    predictions = []
    for future_time in future_times:
        hour = future_time.hour
        minute = future_time.minute
        recent_data = teaching_df.iloc[-1][['ApparentTemperature', 'RelativeHumidity', 'CDD', 'HDD']]
        recent_data['Hour24'] = hour
        recent_data['Minute'] = minute

        features_scaled = scaler.transform([recent_data])
        prediction = nn_model.predict(features_scaled)[0][0]
        predictions.append((future_time, prediction))

    # Generate plot
    plt.figure(figsize=(14, 7))
    times = [time[0].strftime('%H:%M') for time in predictions]
    values = [time[1] for time in predictions]
    plt.plot(times, values, marker='o')

    plt.title(f'Predicted Energy Consumption for the Next {period_minutes} Minutes')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (kWh)')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.xticks(rotation=45)
    plt.show()

    alerts = [pred for time, pred in predictions if pred > ENERGY_USAGE_LIMIT]
    if alerts:
        tips = get_energy_saving_tips()
        alert_times = ', '.join([time.strftime('%H:%M') for time, pred in predictions if pred > ENERGY_USAGE_LIMIT])
        return f"Warning: Energy consumption is predicted to exceed the limit of {ENERGY_USAGE_LIMIT} kWh at {alert_times}.\n\nTo help reduce energy consumption, consider the following tips:\n{tips}"
    else:
        return "Good news! The energy consumption is predicted to be within the safe limit."

def get_energy_saving_tips():
    tips = [
        "Turn off lights when not in use.",
        "Use energy-efficient appliances.",
        "Regularly maintain HVAC systems.",
        "Utilize natural lighting during the day.",
        "Insulate your building properly.",
        "Use programmable thermostats to control heating and cooling.",
        "Ensure computers and other office equipment are turned off when not in use.",
        "Use blinds or shades to control the temperature.",
        "Encourage the use of stairs instead of elevators.",
        "Conduct regular energy audits to identify areas for improvement."
    ]
    return "\n".join(tips)

def chatbot():
    print("Hello! I'm your Energy Consumption Assistant.")
    print("You can ask me about energy consumption predictions, usage patterns, or tips on saving energy.")
    print("Here are some examples of what you can ask me:")
    print("1. 'What were the usage patterns for the teaching building on 2018-10-31?'")
    print("2. 'Can you predict the energy consumption of the teaching building for the next 30 minutes?'")
    print("3. 'How can I lower the energy consumption of the teaching building?'")
    print("Type 'exit' or 'quit' to end the conversation.")

    while True:
        user_input = input("\nYou: ")

        if "usage patterns" in user_input.lower():
            date_str = extract_date_from_input(user_input)
            response = summarize_usage_patterns(date_str) if date_str else summarize_usage_patterns()
        elif "predict" in user_input.lower() and "next" in user_input.lower():
            period_minutes = extract_period_from_input(user_input)
            if period_minutes:
                response = predict_energy_consumption(period_minutes)
            else:
                response = "Please provide a specific period for the prediction (e.g., next 30 minutes)."
        elif "lower the energy consumption" in user_input.lower() or "tips" in user_input.lower():
            response = get_energy_saving_tips()
        elif user_input.lower() in ["exit", "quit"]:
            print("Goodbye! Feel free to ask me any energy-related questions anytime.")
            break
        else:
            response = "I'm sorry, I don't understand that request. Could you please rephrase it?"

        print(f"\nChatbot: {response}")

def extract_date_from_input(user_input):
    # Attempt to extract date using known formats
    if "on" in user_input.lower():
        date_part = user_input.lower().split("on")[1].strip()
        for fmt in ("%Y-%m-%d", "%d-%m-%Y", "%Y/%m/%d", "%d/%m/%Y"):
            try:
                parsed_date = datetime.strptime(date_part, fmt)
                return parsed_date.date()
            except ValueError:
                continue
    return None

def extract_period_from_input(user_input):
    # Dummy implementation to extract the period from the user input
    if "next" in user_input.lower():
        period_part = user_input.lower().split("next")[1].strip().split(" ")[0]
        try:
            return int(period_part)
        except ValueError:
            return None
    return None

if __name__ == "__main__":
    chatbot()

Hello! I'm your Energy Consumption Assistant.
You can ask me about energy consumption predictions, usage patterns, or tips on saving energy.
Here are some examples of what you can ask me:
1. 'What were the usage patterns for the teaching building on 2018-10-31?'
2. 'Can you predict the energy consumption of the teaching building for the next 30 minutes?'
3. 'How can I lower the energy consumption of the teaching building?'
Type 'exit' or 'quit' to end the conversation.

You: How can I lower the energy consumption of the teaching building?

Chatbot: Turn off lights when not in use.
Use energy-efficient appliances.
Regularly maintain HVAC systems.
Utilize natural lighting during the day.
Insulate your building properly.
Use programmable thermostats to control heating and cooling.
Ensure computers and other office equipment are turned off when not in use.
Use blinds or shades to control the temperature.
Encourage the use of stairs instead of elevators.
Conduct regular energy audits to ide